In [1]:
%load_ext sql

In [2]:
import os

USER = os.environ['POSTGRESQL_USER']
PASSWORD = os.environ['POSTGRESQL_PASSWORD']
POSTGRESQL_HOST = '10.129.0.25'
DBASE_NAME = 'demo'

import psycopg2

with psycopg2.connect(
    dbname='jovyan'
) as conn:
    with conn.cursor() as cur:
        cur.execute('SELECT datname FROM pg_database')
        records = cur.fetchall()
        
print(records)

[('postgres',), ('jovyan',), ('template1',), ('template0',)]


In [3]:
%%sql postgresql:///jovyan
CREATE TABLE employees(
    employee_id int NOT NULL,
    first_name varchar(20) NOT NULL,
    last_name varchar(20) NOT NULL,
    hire_date date NOT NULL,
    job_id varchar(20) NOT NULL,
    manager_id int NOT NULL,
    department_id int NOT NULL 
    );

Done.


[]

In [4]:
%%sql postgresql:///jovyan
insert into employees(employee_id, first_name,last_name,hire_date,job_id, manager_id,department_id) values
(100,'Steven', 'King', '1987-06-17','AD_PRES', 0,90),
(101,'Neena', 'Kochhar', '1987-06-18','AD_VP', 100, 90),
(102, 'Lex', 'DeHaan', '1987-06-19', 'AD_VP', 100, 90),
(103, 'Alexander', 'Hunold', '1987-06-20', 'IT_PROG', 102, 60),
(107, 'Diana', 'Lorentz', '1987-06-24', 'IT_PROG', 103, 60),
(108, 'Nancy', 'Greenberg', '1987-06-25', 'FI_MGR', 101, 100),
(109, 'Daniel' ,'Faviet', '1987-06-26', 'FI_ACCOUNT', 108, 100),
(114, 'Den', 'Raphaely', '1987-07-01', 'PU_MAN', 110, 30),
(118, 'Guy', 'Himuro', '1987-07-05', 'PU_CLERK', 114, 30),
(144, 'Peter', 'Vargas', '1987-07-31', 'PU_CLERK', 114, 50),
(145,'John', 'Russell', '1987-08-01', 'ST_CLERK' , 100, 80),
(146, 'Karen', 'Partners', '1987-08-02', 'SA_MAN', 100, 80);

12 rows affected.


[]

In [5]:
%%sql postgresql:///jovyan
select * from employees

12 rows affected.


employee_id,first_name,last_name,hire_date,job_id,manager_id,department_id
100,Steven,King,1987-06-17,AD_PRES,0,90
101,Neena,Kochhar,1987-06-18,AD_VP,100,90
102,Lex,DeHaan,1987-06-19,AD_VP,100,90
103,Alexander,Hunold,1987-06-20,IT_PROG,102,60
107,Diana,Lorentz,1987-06-24,IT_PROG,103,60
108,Nancy,Greenberg,1987-06-25,FI_MGR,101,100
109,Daniel,Faviet,1987-06-26,FI_ACCOUNT,108,100
114,Den,Raphaely,1987-07-01,PU_MAN,110,30
118,Guy,Himuro,1987-07-05,PU_CLERK,114,30
144,Peter,Vargas,1987-07-31,PU_CLERK,114,50


In [6]:
%%sql postgresql:///jovyan
CREATE TABLE departments(
    department_id int NOT NULL,
    department_name varchar(20) NOT NULL,
    manager_id int NOT NULL);

Done.


[]

In [7]:
%%sql postgresql:///jovyan
insert into departments(department_id, department_name, manager_id) values
(10, 'Administration', 200),
(30, 'Purchasing', 114),
(40, 'HumanResources', 203),
(50, 'Shipping', 121),
(60, 'IT', 103),
(70, 'PublicRelations', 204),
(80, 'Sales', 145),
(90, 'Executive', 100),
(100, 'Finance', 108),
(110, 'Accounting', 205)

10 rows affected.


[]

In [8]:
%sql select * from departments

 * postgresql:///jovyan
10 rows affected.


department_id,department_name,manager_id
10,Administration,200
30,Purchasing,114
40,HumanResources,203
50,Shipping,121
60,IT,103
70,PublicRelations,204
80,Sales,145
90,Executive,100
100,Finance,108
110,Accounting,205


In [9]:
%%sql postgresql:///jovyan
select department_name, first_name as manager_name, last_name as manager_surname, s.number_of_employees from employees e
right join (select department_name, count(employee_id) as number_of_employees, max(d.manager_id) as manager_id from departments d
right join employees using(department_id)
group by department_name) as s on e.employee_id = s.manager_id
where first_name is not null
order by department_name

5 rows affected.


department_name,manager_name,manager_surname,number_of_employees
Executive,Steven,King,3
Finance,Nancy,Greenberg,2
IT,Alexander,Hunold,2
Purchasing,Den,Raphaely,2
Sales,John,Russell,2
